In [2]:
!pip install git+https://github.com/huggingface/transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qp8rpg02
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qp8rpg02
  Resolved https://github.com/huggingface/transformers to commit c5f0288bc7d76f65996586f79f69fba8867a0e67
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 800.5 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 47.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import torch
import transformers
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import re

from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.embeddings.openai import OpenAIEmbeddings

import nltk
from nltk.tokenize import sent_tokenize
device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-03-23 15:27:39.348308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 15:27:39.348429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 15:27:39.514080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# imports datas

In [4]:
results_df = pd.read_csv("../datas/passage_date.csv", index_col=[0])

In [5]:
results_df.rename(columns={"contexts": "texte"}, inplace=True)

In [6]:
results_df

,filename,texte
0,Agen_100515.txt,Le 12 11 2019 Cour d appel d Agen chambre soci...
1,Agen_1100752.txt,Le 12 11 2019 Cour d appel d Agen chambre civi...
2,Agen_1613.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
3,Agen_2118.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
4,Agen_21229.txt,Le 12 11 2019 Cour d appel d Agen Audience pub...
...,...,...
765,Versailles_602516.txt,Le 12 11 2019 Cour d appel de Versailles ct003...
766,Versailles_605885.txt,Le 12 11 2019 Cour d appel de Versailles ct008...
767,Versailles_61934.txt,Le 12 11 2019 Cour d appel de Versailles ct008...
768,Versailles_67325.txt,Le 12 11 2019 Cour d appel de Versailles ct008...


In [7]:
results_df = results_df[:2] # juste pour le test

# Let's RAG

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map='auto')
tokenizer = transformers.AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


nltk.download('punkt') # Décommente cette ligne si 'punkt' n'a pas déjà été téléchargé

def preprocess_documents(df_or_series):
    pages_content = []
    
    # Vérifie si l'objet est une Series et le convertit en DataFrame si nécessaire
    if isinstance(df_or_series, pd.Series):
        df = df_or_series.to_frame().transpose()
    else:
        df = df_or_series
    
    for index, row in df.iterrows():
        texte = row["texte"]
        name = row["filename"]
        
        # Découpe le texte en phrases
        phrases = sent_tokenize(texte)
        
        for phrase in phrases:
            texte_doc = Document(page_content=phrase,
                                 metadata={'name': name})
            pages_content.append(texte_doc)
        
    return pages_content

In [ ]:
content = preprocess_documents(results_df)
embedding_model = SentenceTransformerEmbeddings(model_name='multi-qa-distilbert-cos-v1')
faiss_index = FAISS.from_documents(content, embedding_model)
retriever = faiss_index.as_retriever(search_kwargs={'k': 3})

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=2000)

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=mistral_llm,
                                           retriever=retriever,
                                           return_source_documents=True,
                                           verbose=False)

# Predict the date

2 options : 
- to predict the date of the accident, use this query : "Quelle est la date de l'accident?"
- to predict the date of consolidation, use this query : "Quelle est la date de consolidation?"

In [ ]:
query = "Quelle est la date de l'accident?"


if query == "Quelle est la date de l'accident?":
    result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
            On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
            La date de l'accident correspond à la date à laquelle l'accident s'est produit. Elle est toujours quelque part dans le document (généralement au début) sauf dans de très rares cas.
            voici la demande : {query} ATTENTION: Tu me renverras la date au format MM-DD-AAAA
            """,
            "chat_history": []})

elif query == "Quelle est la date de consolidation?":
    result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
                On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
                La date de consolidation est la date à laquelle les blessures de la victime sont devenues stables et ont été déclarées définitives par un médecin. L'information devrait être présente dans la plupart des cas mais parfois elle est soit manquante (tu afficheras la valeur "n.c.") soit non applicable (tu afficheras la valeur "n.a.") si la blessure n'a pas stabilisé avant le décès de la victime.
                voici la demande : {query} """,
                "chat_history": []})


In [ ]:
for element in result["source_documents"]:
    source = f'\n- {element.metadata["name"]}'
    print(source)

In [ ]:
print(result['answer'])